For Google Colab

In [ ]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !nohup ollama serve > output.log 2>&1 &
# !ollama pull gemma3

In [ ]:
# !nohup ollama serve > output.log 2>&1 &

In [ ]:
# !pip install langchain langchainhub chromadb ollama
# !pip install langchain-community langchain-core

For Local Jupyter Notebook

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

# Step 1: Setup LLM
llm = OllamaLLM(model="gemma3:12b")

In [ ]:
# Step 2: Short-Term Memory (ConversationBufferMemory)
short_term_memory = ConversationBufferMemory()

short_term_chain = ConversationChain(
    llm=llm,
    memory=short_term_memory,
    verbose=True
)

In [ ]:
# Interact with short-term memory
print("=== Short-Term Memory Example ===")
short_term_chain.predict(input="My name is Ashiq.")

In [ ]:
short_term_chain.predict(input="What's my name?")

In [ ]:
# Step 3: Long-Term Memory (Using Chroma + RetrieverMemory)
embedding = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(embedding_function=embedding, persist_directory="../chroma_store")
retriever = vectorstore.as_retriever()

long_term_memory = VectorStoreRetrieverMemory(retriever=retriever)

In [ ]:
# Retrieve memory later
docs = retriever.get_relevant_documents("What color do I like?")
print("=== Long-Term Memory Retrieval ===")
for doc in docs:
    print(doc.page_content)

# Step 4: Optional - Use in a chain
long_term_chain = ConversationChain(
    llm=llm,
    memory=long_term_memory,
    verbose=True
)

In [ ]:
# Save long-term memory manually
long_term_memory.save_context({"input": "My favorite color is blue."},
                              {"output": "Got it, blue is your favorite."})

In [ ]:
# Interact
long_term_chain.predict(input="Can you remind me what color I like?")

In [ ]:
short_term_chain.predict(input="Can you recall my name?")  # Should recall "Alex"

In [ ]:
short_term_memory.clear()